In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [14]:
dev= pd.read_csv('devices.csv')
tr1= pd.read_csv('transactions_1.csv')
tr2= pd.read_csv('transactions_2.csv')
tr3= pd.read_csv('transactions_3.csv')
users = pd.read_csv('users.csv')
notif = pd.read_csv('notifications.csv')
trans= pd.concat([tr1, tr2, tr3])#all transactions in one dataframe

#users table

In [15]:
users['brands']=dev['brand']#h sthlh brands apo ta devices is added to users table
#ginontai ta unknown stoixeia tyxaia Android, Apple
import random
def change_unknown_device(brand):
  if  brand== 'Unknown':
    return random.sample(['Android', 'Apple'], 1)[0]
  else:
    return brand
users['brands']=users['brands'].apply(change_unknown_device)


In [16]:
users['age']=2019 - users['birth_year']#sthlh age anti gia birth_year

In [17]:
users['plan']=users['plan'].apply(lambda x: 'PAID' if x!="STANDARD" else x)#h sthlh pairnei pleon standard h paid stoixeia

In [18]:
#handling dates in days
users.created_date=pd.to_datetime(users.created_date)
#users['today']=np.datetime64('2019-06-01')
users['account_duration'] = (np.datetime64('2019-06-01') - users['created_date']).dt.days

In [19]:
#users['month']=pd.DatetimeIndex(users['created_date']).month
#users['year']=pd.DatetimeIndex(users['created_date']).year
#users['duration']=users.apply(lambda x: 18-x['month'] if x['year']==2018 else 6-x['month'], axis=1)

#transactions table
-------------------

In [20]:
trans=trans[trans['transactions_state']=='COMPLETED']#kratame mono ta completed transactions

In [21]:
#vriskoume to pote kathe user kanei kathe toy synallagh kai ypologizoume to std, kai to variance twn hmeromhniwn.
#Oi hmeromhnies ypologizontai se meres apo thn 1/6
tempo=trans[['user_id','created_date']]
tempo.sort_values(['user_id','created_date'],inplace=True)
tempo['created_date']=pd.to_datetime(tempo['created_date'])
tempo['days_from_transaction']=(np.datetime64('2019-06-01') - tempo['created_date']).dt.days
tempo.drop(columns=['created_date'], inplace=True)
tempo=tempo.groupby('user_id').agg(std_of_transactions=('days_from_transaction','std'),var_of_transactions=('days_from_transaction','var'))


c:\users\thrash\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
c:\users\thrash\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
c:\users\thrash\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value i

In [22]:
users=pd.merge(users , tempo, how='left', left_on='user_id', right_index=True)

In [23]:
number_of_trans=pd.DataFrame(trans.user_id.value_counts())#amount of transactions per user
number_of_trans.rename(columns={'user_id':'total_amount_of_transactions'},inplace=True)
users=pd.merge(users , number_of_trans, how='left', left_on='user_id', right_index=True)#h sthlg amount_of_transactions is joined sto users

In [24]:
trans_out=trans[trans['direction']=='OUTBOUND']
trans_in=trans[trans['direction']=='INBOUND']

In [25]:
usd_per_user_out = pd.pivot_table(data=trans_out,aggfunc=['count','min','max','mean','sum'], index='user_id',  values='amount_usd')
usd_per_user_out.columns=['amount_of_transactions_out', 'min_usd_out', 'max_usd_out', 'mean_usd_out', 'sum_usd_out']
users=pd.merge(users , usd_per_user_out, how='left', left_on='user_id', right_index=True)

In [26]:
usd_per_user_in = pd.pivot_table(data=trans_in,aggfunc=['count','min','max','mean','sum'], index='user_id',  values='amount_usd')
usd_per_user_in.columns=['amount_of_transactions_in', 'min_usd_in', 'max_usd_in', 'mean_usd_in', 'sum_usd_in']
users=pd.merge(users , usd_per_user_in, how='left', left_on='user_id', right_index=True)

In [27]:
#pd.crosstab(trans.transactions_type, trans.ea_merchant_country)

In [28]:
#pd.crosstab(trans.transactions_type, trans.ea_cardholderpresence)

In [29]:
trans_transactions_type_out = pd.crosstab(trans_out['user_id'], columns=trans_out['transactions_type'])
trans_transactions_type_in = pd.crosstab(trans_in['user_id'], columns=trans_in['transactions_type'])
trans_transactions_type_out.columns=['count_type_atm', 'count_type_card_payment','count_type_exchange','count_type_fee', 'count_type_tax', 'count_type_transfer_out']
trans_transactions_type_in.columns=['count_type_card_refund', 'count_type_cashback','count_type_refund','count_type_topup','count_type_transfer_in']

In [30]:
users=pd.merge(users , trans_transactions_type_out, how='left', left_on='user_id', right_index=True)

In [31]:
users=pd.merge(users , trans_transactions_type_in, how='left', left_on='user_id', right_index=True)

In [32]:
#trans['month']=pd.DatetimeIndex(trans['created_date']).month#created month and year as columns
#trans['year']=pd.DatetimeIndex(trans['created_date']).year


In [33]:
trans_sorted=trans.sort_values(['created_date'],ascending=False)#trans is sorted, duplicates user_id entries are removed and remain only latest transactions 
lasttrans = trans_sorted.drop_duplicates('user_id').sort_index()
lasttrans=lasttrans[['user_id','created_date']]

In [34]:
#handling dates in days
lasttrans['created_date']=pd.to_datetime(lasttrans['created_date'])
#lasttrans['today']=np.datetime64('2019-06-01')
lasttrans['last_transaction'] = (np.datetime64('2019-06-01') - lasttrans['created_date']).dt.days

In [35]:
#lasttrans['month']=pd.DatetimeIndex(lasttrans['created_date']).month
#lasttrans['year']=pd.DatetimeIndex(lasttrans['created_date']).year
#lasttrans['months_from_last_transaction']=lasttrans.apply(lambda x: 17-x['month'] if x['year']==2018 else 5-x['month'], axis=1)

In [36]:
users=pd.merge(users , lasttrans, how='left', left_on='user_id', right_on='user_id')

In [37]:
trans_sorted_asc=trans.sort_values(['created_date'],ascending=True)#trans is sorted, duplicates user_id entries are removed and remain only latest transactions 
firsttrans = trans_sorted_asc.drop_duplicates('user_id').sort_index()
firsttrans=firsttrans[['user_id','created_date']]

In [38]:
#handling dates in days
firsttrans['created_date']=pd.to_datetime(firsttrans['created_date'])
#firsttrans['today']=np.datetime64('2019-06-01')
firsttrans['first_transaction'] = (np.datetime64('2019-06-01')-firsttrans['created_date']).dt.days

In [39]:
#lasttrans['month']=pd.DatetimeIndex(lasttrans['created_date']).month
#lasttrans['year']=pd.DatetimeIndex(lasttrans['created_date']).year
#lasttrans['months_from_first_transaction']=lasttrans.apply(lambda x: 18-x['month'] if x['year']==2018 else 6-x['month'], axis=1)

In [40]:
users=pd.merge(users , firsttrans, how='left', left_on='user_id', right_on='user_id')

In [41]:
users.sample(4)

,user_id,birth_year,country,city,created_date_x,user_settings_crypto_unlocked,plan,attributes_notifications_marketing_push,attributes_notifications_marketing_email,num_contacts,...,count_type_transfer_out,count_type_card_refund,count_type_cashback,count_type_refund,count_type_topup,count_type_transfer_in,created_date_y,last_transaction,created_date,first_transaction
14251,user_14251,1987,GB,Belfast,2018-08-11 08:06:56.289242,0,STANDARD,1.0,1.0,0,...,2.0,0.0,0.0,0.0,2.0,0.0,2018-12-02 08:13:04.732242,180.0,2018-08-31 11:50:19.109242,273.0
6793,user_6793,1989,GB,London,2018-11-28 07:18:08.043043,0,STANDARD,NaN,NaN,0,...,7.0,0.0,0.0,0.0,26.0,0.0,2019-05-10 13:01:04.847043,21.0,2018-11-29 04:16:06.014043,183.0
9320,user_9320,1999,FR,Limoges,2018-12-13 10:38:14.144749,0,STANDARD,NaN,NaN,13,...,5.0,0.0,0.0,0.0,1.0,0.0,2019-01-14 19:59:59.805749,137.0,2018-12-14 04:38:50.777749,168.0
12562,user_12562,1950,GB,Washington,2018-05-26 07:39:26.518877,0,STANDARD,NaN,NaN,0,...,0.0,0.0,0.0,0.0,1.0,0.0,2018-05-26 07:49:31.742877,370.0,2018-05-26 07:45:04.544877,370.0


In [42]:
users['engagement_duration']=users['first_transaction'] - users['last_transaction'] + 1

In [43]:
users.columns

Index(['user_id', 'birth_year', 'country', 'city', 'created_date_x',
       'user_settings_crypto_unlocked', 'plan',
       'attributes_notifications_marketing_push',
       'attributes_notifications_marketing_email', 'num_contacts',
       'num_referrals', 'num_successful_referrals', 'brands', 'age',
       'account_duration', 'std_of_transactions', 'var_of_transactions',
       'total_amount_of_transactions', 'amount_of_transactions_out',
       'min_usd_out', 'max_usd_out', 'mean_usd_out', 'sum_usd_out',
       'amount_of_transactions_in', 'min_usd_in', 'max_usd_in', 'mean_usd_in',
       'sum_usd_in', 'count_type_atm', 'count_type_card_payment',
       'count_type_exchange', 'count_type_fee', 'count_type_tax',
       'count_type_transfer_out', 'count_type_card_refund',
       'count_type_cashback', 'count_type_refund', 'count_type_topup',
       'count_type_transfer_in', 'created_date_y', 'last_transaction',
       'created_date', 'first_transaction', 'engagement_duration'],
      

In [44]:
users['days_passed_for_first_transaction']= users['account_duration']- users['first_transaction'] 

In [45]:
users.sample()

,user_id,birth_year,country,city,created_date_x,user_settings_crypto_unlocked,plan,attributes_notifications_marketing_push,attributes_notifications_marketing_email,num_contacts,...,count_type_cashback,count_type_refund,count_type_topup,count_type_transfer_in,created_date_y,last_transaction,created_date,first_transaction,engagement_duration,days_passed_for_first_transaction
8319,user_8319,1996,BG,Sofiya,2018-07-28 02:35:29.454939,1,STANDARD,1.0,1.0,26,...,0.0,0.0,18.0,44.0,2019-05-10 05:52:33.003939,21.0,2018-09-21 04:30:29.789939,252.0,232.0,55.0


In [46]:
users['transactions_frequency']=users['total_amount_of_transactions'] / users['engagement_duration']

#Notifications
________________________________________________________


In [47]:
notif_sent=notif[notif['status']=='SENT']


In [48]:
amount_notif_sent=pd.crosstab(notif_sent['user_id'], columns=[notif_sent['channel'],notif_sent['reason']])
amount_notif_sent.columns=['_'.join(col).lower() for col in amount_notif_sent.columns.values]

In [49]:
users=pd.merge(users, amount_notif_sent, how='left', left_on='user_id', right_index=True)


###notifications table analysis under process

In [50]:
#pd.crosstab(notif['reason'], columns=notif['status'])

In [51]:
#pd.crosstab(notif['user_id'], columns=[notif['channel'], notif['status'], notif['reason']]).head(5)

In [52]:
#notif[notif['user_id']=='user_5734'].sort_values('created_date')#Z

In [53]:
#users[users['user_id']=='user_12544']

In [54]:
#=pd.crosstab(notif['user_id'], columns= notif['reason'])
#tepm2.sample(5)

In [55]:
#=pd.merge(users, tepm2, left_on='user_id', right_index=True )

In [56]:
#=temp[['plan','BLACK_FRIDAY','BLUE_TUESDAY','ENGAGEMENT_SPLIT_BILL_RESTAURANT','ENGAGEMENT_SPLIT_BILL_RESTAURANT','INVEST_IN_GOLD','JOINING_ANNIVERSARY','LOST_CARD_ORDER','MADE_MONEY_REQUEST_NOT_SPLIT_BILL','METAL_RESERVE_PLAN','NO_INITIAL_CARD_ORDER','NO_INITIAL_CARD_USE','ONBOARDING_TIPS_ACTIVATED_USERS','PROMO','PROMO_CARD_ORDER','REENGAGEMENT_ACTIVE_FUNDS','SILVER_ENGAGEMENT_FEES_SAVED','SILVER_ENGAGEMENT_INACTIVE_CARD','WELCOME_BACK']]

In [57]:
#temp[temp['plan']=='PAID'].sample(50)

#Dropping and encoding users table

In [59]:
#dropping columns we don't need
data=users.drop(columns=['user_id','birth_year','city','created_date_x','attributes_notifications_marketing_push','attributes_notifications_marketing_email','num_referrals','num_successful_referrals','created_date_y','created_date'])


#'year_x','month_x','month_y','year_y','month','year'

In [60]:
data.columns

Index(['country', 'user_settings_crypto_unlocked', 'plan', 'num_contacts',
       'brands', 'age', 'account_duration', 'std_of_transactions',
       'var_of_transactions', 'total_amount_of_transactions',
       'amount_of_transactions_out', 'min_usd_out', 'max_usd_out',
       'mean_usd_out', 'sum_usd_out', 'amount_of_transactions_in',
       'min_usd_in', 'max_usd_in', 'mean_usd_in', 'sum_usd_in',
       'count_type_atm', 'count_type_card_payment', 'count_type_exchange',
       'count_type_fee', 'count_type_tax', 'count_type_transfer_out',
       'count_type_card_refund', 'count_type_cashback', 'count_type_refund',
       'count_type_topup', 'count_type_transfer_in', 'last_transaction',
       'first_transaction', 'engagement_duration',
       'days_passed_for_first_transaction', 'transactions_frequency',
       'email_engagement_split_bill_restaurant', 'email_invest_in_gold',
       'email_lost_card_order', 'email_made_money_request_not_split_bill',
       'email_no_initial_card_orde

In [61]:
data.isnull().sum()# ta nan tha ginoyn 0, afoy einai eite counts eite sum klp,kai 
datatest=data.fillna(0)#dhmioyrgeitai o datatest gia xrhsh se modela

In [62]:
#column plan  encoding sto dataset
plan_dict = {'STANDARD': 0, 'PAID': 1}
datatest['plan'] = datatest.plan.map(plan_dict)



In [63]:
datatest.drop('country', axis=1, inplace=True)
#gia thn wra petaw ta countries mexri na doyme pws ua ta xeiristoume

In [64]:
#column brands encoding sto dataset
datatest=pd.get_dummies(datatest)

In [65]:
#dhmioyrgoyntai oi pinakes pou exw anevasei
datatest.to_csv('std_for_testing.csv',index=False, sep=';')
data.to_csv('std_from_users.csv',index=False, sep=';')
users.to_csv('std_users_feat.csv',index=False, sep=';')

In [66]:
datatest[datatest['transactions_frequency']==np.inf]

,user_settings_crypto_unlocked,plan,num_contacts,age,account_duration,std_of_transactions,var_of_transactions,total_amount_of_transactions,amount_of_transactions_out,min_usd_out,...,push_joining_anniversary,push_lost_card_order,push_promo,push_reengagement_active_funds,push_silver_engagement_fees_saved,push_silver_engagement_inactive_card,sms_metal_reserve_plan,sms_reengagement_active_funds,brands_Android,brands_Apple


In [68]:
datatest.sample(5)

,user_settings_crypto_unlocked,plan,num_contacts,age,account_duration,std_of_transactions,var_of_transactions,total_amount_of_transactions,amount_of_transactions_out,min_usd_out,...,push_joining_anniversary,push_lost_card_order,push_promo,push_reengagement_active_funds,push_silver_engagement_fees_saved,push_silver_engagement_inactive_card,sms_metal_reserve_plan,sms_reengagement_active_funds,brands_Android,brands_Apple
861,0,0,8,26,360,108.498441,11771.911694,538.0,420.0,0.07,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0,1
6396,0,0,67,30,415,104.088720,10834.461545,106.0,83.0,1.34,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1
8916,0,0,14,41,189,37.476659,1404.500000,2.0,0.0,0.00,...,0.0,1.0,1.0,4.0,0.0,0.0,0.0,0.0,1,0
5770,0,0,24,31,303,78.259727,6124.584795,19.0,17.0,2.40,...,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,1,0
345,1,0,6,33,269,37.651780,1417.656531,62.0,51.0,0.11,...,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0,1


In [70]:
datatest[datatest['std_of_transactions']==0]

,user_settings_crypto_unlocked,plan,num_contacts,age,account_duration,std_of_transactions,var_of_transactions,total_amount_of_transactions,amount_of_transactions_out,min_usd_out,...,push_joining_anniversary,push_lost_card_order,push_promo,push_reengagement_active_funds,push_silver_engagement_fees_saved,push_silver_engagement_inactive_card,sms_metal_reserve_plan,sms_reengagement_active_funds,brands_Android,brands_Apple
5,0,0,0,29,275,0.0,0.0,2.0,1.0,20.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1
16,0,0,5,24,300,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0,1
26,0,0,0,38,234,0.0,0.0,3.0,2.0,1.99,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1,0
34,0,0,0,43,262,0.0,0.0,1.0,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0
54,0,0,5,34,205,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15475,0,0,0,36,354,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1
15487,0,0,0,36,346,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0
15494,0,0,0,68,354,0.0,0.0,2.0,1.0,4.99,...,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0,1
15512,0,0,5,53,205,0.0,0.0,1.0,1.0,4.99,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0


In [67]:
#dhmioyrgia X, y pinakwn gia na etoimasia toy train_test_split
#X = us_1_en.drop(columns=['plan']).values
#X_columns = us_1_en.drop(columns=['plan']).columns
#y = us_1_en['plan'].values